In [1]:
!pip install -q -U bitsandbytes
    
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
    
!pip install -q -U huggingface_hub
!pip install -q -U langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf

In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['DF_TOKEN'] = user_secrets.get_secret("DF_TOKEN")

In [3]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
!pip install langchain_community
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline

2024-05-28 08:51:31.773457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 08:51:31.773549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 08:51:31.955554: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s eta 0:00:0000:0100:01


In [4]:
class ConversationAssistant:
    def __init__(self, llm_model) -> None:
        self.quant = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(llm_model)
        self.model = AutoModelForCausalLM.from_pretrained(
            llm_model, 
            quantization_config=self.quant, low_cpu_mem_usage=True) 

In [8]:
from huggingface_hub import login
login()

In [6]:
# !pip install bitsandbytes =0.39.0

In [9]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
assistant = ConversationAssistant(model_name)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [10]:
text_generation_pipeline = pipeline(
    model=assistant.model,
    tokenizer=assistant.tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
    
)

In [11]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/kaggle/input/health-book/healthcare_book.pdf")
pages = loader.load_and_split()
pages[0]

Document(page_content='123Gørill Haugan\nMonica Eriksson\nEditorsHealth Promotion in \nHealth Care – Vital \nTheories and Research', metadata={'source': '/kaggle/input/health-book/healthcare_book.pdf', 'page': 0})

In [14]:
pip install rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 83.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
loader = PyPDFLoader("/kaggle/input/health-book/healthcare_book.pdf", extract_images=True)
pages = loader.load()
pages[4].page_content

'v\n High Ages: A\xa0Success That Signifies Health Challenges\nAs we all are aware of, there is a demographic shift toward an older popula-\ntion worldwide. The segment of people aged 80\xa0years and more is growing \nrapidly and those aged 100+ are growing fastest [1]. This is a sign that we \nhave succeeded in some ways. This shift started in high-income countries. \nHowever, for the first time in history, people can expect to live until their 60s \nand beyond globally [2]. During the period from 2015 and 2050, the propor -\ntion of those 60\xa0years and more will nearly double from 12 to 22%; by 2050 \nindividuals aged 60\xa0years and older is expected to total two billion, up from \n900\xa0 million in 2015 [2]. Currently, low- and middle-income countries are \nexperiencing the greatest demographic change; hence, all countries are now \nfacing major challenges to ensure that their health and social systems are \nready to make the most of this demographic shift [2, 3]. Age is an aim 

In [16]:
# 2. Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(pages)

In [17]:
!pip install -q -U sentence-transformers

In [18]:
# 3. Define the embedding model

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
!pip install -q -U faiss-gpu

In [20]:
# 4. Create the vector store 
vector = FAISS.from_documents(documents, embeddings)

In [21]:
# 5. Define a retriever interface
retriever = vector.as_retriever()

In [22]:
# 6. Define LLM
model = HuggingFacePipeline(pipeline=text_generation_pipeline)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [24]:
# Define prompt template
prompt = ChatPromptTemplate.from_template(
"""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [25]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# response = model.generate(prompt, max_length=50, do_sample=True)


response = retrieval_chain.invoke({"input": "What are contributing factors of heart failure?"})

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
# Define the question
question = "What are contributing factors of heart failure?"

# Using a try-except block to handle potential errors during the invocation
try:
    # Invoke the retrieval chain with the question
    response = retrieval_chain.invoke({"input": question})
    
    # Check if the response contains the expected output, assuming it's a dictionary
    if 'output' in response:
        # Process or print the response output
        print("Answer:", response['output'])
    else:
        # Handle cases where the response might not contain the expected output
        print("No valid output received from the retrieval chain.")
except Exception as e:
    # Print the error message if something goes wrong during the invocation
    print("An error occurred:", str(e))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


No valid output received from the retrieval chain.


In [27]:
print(response["answer"])

Human: Answer the following question based only on the provided context:

<context>
283
 69. Chaudhry SI, Wang Y , Concato J, Gill TM, 
Krumholz HM.  Patterns of weight change preced-
ing hospitalization for heart failure. Circulation. 
2007;116(14):1549–54.
 70. Riegel B, Lee CS, Dickson VV , Carlson B. An update 
on the self-care of heart failure index. J Cardiovasc 
Nurs. 2009;24(6):485–97.
 71. Malhotra C, Cheng Sim Wong G, Tan BC, Ng CSH, 
Lee NC, Lau CSL, et  al. Living with heart fail-
ure: perspectives of patients from Singapore. Proc 
Singapore Healthc. 2016;25(2):92–7.
 72. Clark AM, Spaling M, Harkness K, Spiers J, 
Strachan PH, Thompson DR, et al. Determinants of 
effective heart failure self-care: a systematic review 
of patients’ and caregivers’ perceptions. Heart. 
2014;100(9):716–21.
 73. Costello JA, Boblin S.  What is the experience of 
men and women with congestive heart failure? 
Can J Cardiovasc Nurs. 2004;14(3):9–20. https://
www.scopus.com/inward/record.uri?eid=2